# **Content Based Movie Recommender System**

## **Business Problem**

### Description:

we can use popularity, release date, revenue, vote average and vote count, 

'credit_score','age','tenure','products_number','credit_card','active_member','churn'

Source:https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata/code

In [60]:
#importing the necessary packages
import numpy as np
import pandas as pd

In [61]:
# importing the data
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [62]:
#preview of the movies dataframe
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [63]:
movies.shape

(4803, 20)

In [64]:
#preview of the credits dataframe
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [65]:
credits.shape

(4803, 4)

In [66]:
#Merging both the dataframes into movie_credit dataframe
movie_credit = movies.merge(credits,on='title')

In [67]:
#preview of the movie_credit dataframe
movie_credit.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [68]:
movie_credit.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'cast', 'crew'],
      dtype='object')

We can see there are different languages are present in the original tittle, hence we will not consider.

Tagline is loosly related to the movies. Hence we shall not consider.

In [69]:
#Considering the only columns needed to create our tag
movie_credit = movie_credit[['id','title','overview','genres','keywords','cast','crew']]
movie_credit.head(1)

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


Droped the rest of the columns like release date, popularity, vote count as they are numerical in nature and not needed for the tag based approach we make. These features can be incorporated as an additional feature in the future. Thelanguage column is highly biased as 95% values are en and the rest are differnt countries.

In [70]:
#checking the missing values in the dataframe
movie_credit.isnull().sum()

id          0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

we can see the overview column has 3 missing values

In [71]:
movie_credit[movie_credit['overview'].isnull()]

,id,title,overview,genres,keywords,cast,crew
2662,370980,Chiamatemi Francesco - Il Papa della gente,NaN,"[{""id"": 18, ""name"": ""Drama""}]","[{""id"": 717, ""name"": ""pope""}, {""id"": 5565, ""na...","[{""cast_id"": 5, ""character"": ""Jorge Mario Berg...","[{""credit_id"": ""5660019ac3a36875f100252b"", ""de..."
4147,459488,"To Be Frank, Sinatra at 100",NaN,"[{""id"": 99, ""name"": ""Documentary""}]","[{""id"": 6027, ""name"": ""music""}, {""id"": 225822,...","[{""cast_id"": 0, ""character"": ""Narrator"", ""cred...","[{""credit_id"": ""592b25e4c3a368783e065a2f"", ""de..."
4437,292539,Food Chains,NaN,"[{""id"": 99, ""name"": ""Documentary""}]",[],[],"[{""credit_id"": ""5470c3b1c3a368085e000abd"", ""de..."


In [72]:
#movie_credit['overview'] has space as a value eg:4407 row
#movie_credit[movie_credit['cast']=='[]']     genres	keywords	cast	crew has '[]' as values


In [73]:
#dropped the null value as it is significantly smaller
movie_credit.dropna(inplace=True)
movie_credit.isnull().sum()

id          0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [74]:
#checking for duplicated rows
movie_credit.duplicated().sum()

0

In [75]:
movie_credit.loc[0,'genres']

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [76]:
#Fetching only the genres
import ast

def convert(obj):
    l=[]
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l


In [77]:
movie_credit['genres'] = movie_credit['genres'].apply(convert)
movie_credit['keywords'] = movie_credit['keywords'].apply(convert)

In [78]:
movie_credit.head(1)

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [79]:
#Obtaining the top3 cast members
def cast(obj):
    l=[]
    count = 0
    for i in ast.literal_eval(obj):
        if count<3:
            l.append(i['name'])
            count+=1
        continue
    return l

In [80]:
movie_credit['cast'] = movie_credit['cast'].apply(cast)

In [81]:
movie_credit.head(1)

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [82]:
movie_credit['crew'][0]

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [83]:
#Obtaining the director of the movie
def crew(obj):
    l=[]
    for i in ast.literal_eval(obj):
        if i['job'] == "Director":
            l.append(i['name'])
            break
    return l


In [84]:
movie_credit['crew'] = movie_credit['crew'].apply(crew)

In [85]:
movie_credit.head(1)

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]


In [86]:
movie_credit['overview'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [87]:
movie_credit['overview'] = movie_credit['overview'].apply(lambda x:x.split())

In [88]:
movie_credit.head(1)

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]


In [89]:
movie_credit['genres'] = movie_credit['genres'].apply(lambda x:[i.replace(' ','') for i in x])
movie_credit['keywords'] = movie_credit['keywords'].apply(lambda x:[i.replace(' ','') for i in x])
movie_credit['cast'] = movie_credit['cast'].apply(lambda x:[i.replace(' ','') for i in x])
movie_credit['crew'] = movie_credit['crew'].apply(lambda x:[i.replace(' ','') for i in x])

In [90]:
movie_credit.head()

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]


In [91]:
movie_credit['tags'] = movie_credit['overview'] + movie_credit['genres'] + movie_credit['keywords'] + movie_credit['cast'] + movie_credit['crew']

In [92]:
movie_credit.head()

,id,title,overview,genres,keywords,cast,crew,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes],"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan],"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton],"[John, Carter, is, a, war-weary,, former, mili..."


In [93]:
movie_credit['tags'] = movie_credit['tags'].apply(lambda x : " ".join(x))

In [94]:
movie_credit.head()

,id,title,overview,genres,keywords,cast,crew,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes],A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan],Following the death of District Attorney Harve...
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton],"John Carter is a war-weary, former military ca..."


In [95]:
new_df = movie_credit[['id', 'title', 'tags']]

In [96]:
new_df.head()

,id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [97]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

C:\Users\Vishal\AppData\Local\Temp\ipykernel_9704\3214958533.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())


In [98]:
new_df.head()

,id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."


In [39]:
import nltk

In [40]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [41]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [42]:
new_df['tags'] = new_df['tags'].apply(stem)

C:\Users\Vishal\AppData\Local\Temp\ipykernel_14384\3213734980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [43]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [44]:
vector = cv.fit_transform(new_df['tags']).toarray()

In [45]:
vector

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [46]:
cv.get_feature_names()

C:\Users\Vishal\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['16thcenturi',
 '17thcenturi',
 '18thcenturi',
 '1910',
 '1930',
 '1940',
 '1950',
 '1960',
 '1970',
 '1980',
 '1990',
 '19thcenturi',
 '21stcenturi',
 '3d',
 'aaroneckhart',
 'aarontaylor',
 'abbiecornish',
 'abhishekbachchan',
 'abigailbreslin',
 'aborigin',
 'abram',
 'absurd',
 'abus',
 'abusivehusband',
 'accid',
 'action',
 'actionhero',
 'activist',
 'actor',
 'actress',
 'adambeach',
 'adambrodi',
 'adamcarolla',
 'adamdriv',
 'adamgarcia',
 'adamgoldberg',
 'adammckay',
 'adamrifkin',
 'adamsandl',
 'adamscott',
 'adamshankman',
 'adapt',
 'addict',
 'adolesc',
 'adolfhitl',
 'adopt',
 'adoptedchild',
 'adoptivefath',
 'adrianlyn',
 'adrienbrodi',
 'adriennebarbeau',
 'adultanim',
 'adulteri',
 'advanc',
 'adventur',
 'adversari',
 'advertisingexecut',
 'advertisingexpert',
 'advic',
 'affair',
 'affect',
 'afghanistan',
 'africa',
 'africanamerican',
 'aftercreditssting',
 'afterlif',
 'age',
 'agediffer',
 'agent',
 'aggressionbyanim',
 'aid',
 'aidanquinn',
 'aimeegarcia',

In [47]:
from sklearn.metrics.pairwise import cosine_similarity

In [48]:
similarity =  cosine_similarity(vector)

In [49]:
new_df.loc[3730,'title']

'Anne of Green Gables'

In [50]:
sorted(list(enumerate(similarity[0])),reverse = True,key = lambda x:x[1])[1:6]

[(47, 0.3849001794597506),
 (2378, 0.34426518632954817),
 (61, 0.3333333333333334),
 (83, 0.3142696805273545),
 (2409, 0.30123203803835463)]

In [51]:
def recommend(movie):
    movie_index = new_df[new_df["title"] == movie].index[0]
    movie_list = sorted(list(enumerate(similarity[movie_index])),reverse = True,key = lambda x:x[1])[1:6]
    
    for i in movie_list:
        recom5 = new_df.iloc[i[0]].title
        print(recom5)

In [52]:
recommend('Batman Begins')

The Dark Knight
The Dark Knight Rises
Amidst the Devil's Wings
Mi America
Ajami


In [53]:
new_df.iloc[47,1]

'Star Trek Into Darkness'

In [54]:
import pickle

In [55]:
pickle.dump(new_df,open('movies.pkl','wb'))

In [57]:
pickle.dump(similarity,open('similarity.pkl','wb'))